# Structural Collection

In [5]:
import os
import re
import pickle
import time
import numpy as np
from IndexerCACM import *
from RelevantParser import *
from Query import *
from IRmodel import *
from Vector import *
from IRList import *
from EvalMeasure import *
from EvalIRModel import *
from copy import *

### Get Indexers

In [6]:
# Processed collections 
collectionPath = 'data/cacm/cacm.txt'
collectionPath2 = 'data/cisi/cisi.txt'
queriesPath = 'data/cacm/cacm.qry'
relevantPath = 'data/cacm/cacm.rel'

In [9]:
indexer = IndexerCACM(collectionPath, ParserCACM())

# If Index and Inv Index aren't already builded
#indexer.createRepIndex()
#indexer.createRepInvIndex()
#indexer.createRepInvFromAll()
indexer.createLinkIndex()

queriesIndexer = IndexerCACM(queriesPath, ParserCACM())

# If Index isn't already builded
#queriesIndexer.createRepIndex()

relevantIndexer = Indexer(relevantPath, RelevantParser())

# If Index and Inv Index aren't already builded
#relevantIndexer.createIndex()

### Get Links

In [20]:
print(indexer.getLinkFromDoc(4026))

['1213' '1437' '1562' '1661' '171' '2143' '2336' '2509' '2518' '2983' '532'
 '65' '997']


In [21]:
q = query(1, queriesIndexer, relevantIndexer)
q2 = query(2, queriesIndexer, relevantIndexer)
q3 = query(3, queriesIndexer, relevantIndexer)
q4 = query(4, queriesIndexer, relevantIndexer)
print(q, q2, q3, q4)

(<Query.Query object at 0x7f26683e9bd0>, <Query.Query object at 0x7f263a517050>, <Query.Query object at 0x7f2647848cd0>, <Query.Query object at 0x7f263a573090>)


## Enhanced Search Model

In [22]:
class EnhancedIRmodel(IRmodel):
    
    def __init__(self, indexer, seedModel, n=10, k=1):
        
        IRmodel.__init__(self, indexer)
        
        self.seedModel = seedModel
        self.n = n
        self.k = k
        
    def enlarge(self, graph):

        self.graph = copy(graph)
        goTo = {}

        for node in self.indexer.index:
            links = self.indexer.getLinkFromDoc(node)
            if links[0] != '':
                for id in links:
                    if node in graph:
                        self.graph[id] = 1
                    if id not in goTo:
                        goTo[id] = [node]
                    else:
                        goTo[id].append(node)

        for node in graph:
            if node in goTo:
                for id in np.random.permutation(goTo[node])[:self.k]:
                    self.graph[id] = 1

        i = 0
        self.idToInt = {}
        self.intToId = {}
        for id in self.graph:
            self.idToInt[id] = i
            self.intToId[i] = id
            i += 1

        graphSize = len(self.graph)
        self.links = np.zeros((graphSize, graphSize))

        for id2 in range(graphSize):
            links = self.indexer.getLinkFromDoc(self.intToId[id2])
            if links[0] != '':
                for link in links:
                    if link in self.idToInt:
                        id1 = self.idToInt[link]
                        self.links[id1,id2] = 1
                        
    def getSeeds(self, query):
        
        seeds = {}
        
        idSeeds = self.seedModel.getRanking(query)[:self.n]['id']
        for id in idSeeds:
            seeds[id] = 1
        
        self.enlarge(seeds)
                
    def randomWalk(self):
                
        raise ValueError('Abstract method')
        
    def getScores(self, query):
        
        self.getSeeds(query)
        
        return self.randomWalk()
        

## PageRank Algorithm

Random walk :
Pages and Links represented by a graph $G = (V,E)$
1. Let $p_t$ the current page at time $t$.

2. $\begin{cases} \text{With prob }d\text{ we click on a link of }p_t \text{ or we go randomly if there is no link} \\
\text{With prob }1-d\text{ we go randomly on another webpage}\end{cases}$

We have :

$\mu^{t+1} = \frac{1-d}{N}+dA\mu^{t} \text{ with } A_{i,j} = 
\begin{cases} \frac{1}{N} \text{ if } l_j = 0 \\
\frac{1}{l_j} \text{ if } j\in P_i\\
0 \text{ otherwise}
\end{cases}$

In [27]:
class PageRank(EnhancedIRmodel):
    
    def __init__(self, indexer, seedModel, n=3000, k=500, d=0.5, cvg=10e-20):
        
        EnhancedIRmodel.__init__(self, indexer, seedModel=seedModel, n=n, k=k)
        self.d = d
        self.cvg = cvg
        
    def randomWalk(self):
        
        mu = []
        graphSize = len(self.graph)
            
        mu.append(np.zeros(graphSize) + (1-self.d)/graphSize)
        A = np.zeros((graphSize, graphSize))
        
        for id2 in range(graphSize):
            nLink = self.links[:,id2].sum()
            if nLink==0:
                A[:,id2] = 1/float(graphSize)
            else:
                for id1 in range(graphSize):
                    if self.links[id1, id2]==1:
                        A[id1, id2] = 1/nLink
        
        while(True):
            mu.append((1-self.d)/float(graphSize) + self.d*np.dot(A, mu[-1]))
            if np.linalg.norm(mu[-2]-mu[-1])<self.cvg: break
        
        scores = np.zeros(self.nDoc, [('id', 'a25'), ('score', 'float64')])
        i = 0
        for id in self.indexer.index:
            scores[i]['id'] = id
            if id in self.idToInt:
                scores[i]['score'] = mu[-1][self.idToInt[id]]
            else:
                scores[i]['score'] = 0
            i += 1
            
        return scores


In [26]:
vector = Vector(indexer)
em = PageRank(indexer, vector)
scores = em.getRanking(q2.el)
print(scores[:10])

15.6417448521
[('1781', 0.0016822435371162308) ('1491', 0.0010995973044106281)
 ('1945', 0.0010370644180610576) ('196', 0.0010346694474851021)
 ('3184', 0.0010306799658475908) ('1265', 0.0008862635588820515)
 ('1787', 0.0008639875004658949) ('1860', 0.0008548099934897967)
 ('1751', 0.0008370815198640582) ('2017', 0.0008111151236692741)]


## Hits Algorithm

Pages and Links represented by a graph $G = (V,E)$, at step $t$ :

**autority nodes : ** $a^{t}_{i} = \sum_{j,i\rightarrow j\in E}a_{j}^{t-1} / ||a^{t-1}||_2$

**hub nodes : ** $h^{t}_{i} = \sum_{j,j\rightarrow i\in E}h_{j}^{t-1} / ||h^{t-1}||_2$

In [44]:
class Hits(EnhancedIRmodel):
    
    def __init__(self, indexer, seedModel, n=5, k=5, cvg=10e-6):
        
        EnhancedIRmodel.__init__(self, indexer, seedModel=seedModel, n=n, k=k)
        self.cvg = cvg
        
    def randomWalk(self):
        
        autority = []
        hub = []
        graphSize = len(self.graph)
        autority.append(np.zeros(graphSize) + 1)
        hub.append(np.zeros(graphSize) + 1)
        A = self.links 
        B = np.transpose(A)
        
        while(True):
            autority.append(np.dot(A, hub[-1]))
            autority[-1] = autority[-1]/float(np.linalg.norm(autority[-1]))
            hub.append(np.dot(B, autority[-1]))
            hub[-1] = hub[-1]/float(np.linalg.norm(hub[-1]))
            if np.linalg.norm(autority[-2]-autority[-1])<self.cvg \
                and np.linalg.norm(hub[-2]-hub[-1])<self.cvg: break
        
        scores = np.zeros(self.nDoc, [('id', 'a25'), ('score', 'float64')])
        i = 0
        for id in self.indexer.index:
            scores[i]['id'] = id
            if id in self.idToInt:
                scores[i]['score'] = autority[-1][self.idToInt[id]]
            else:
                scores[i]['score'] = 0
            i += 1
            
        return scores


In [45]:
vector = Vector(indexer)
em = Hits(indexer, vector)
scores = em.getRanking(q2.el)
print(scores[:10])

[('2714', 0.49806072643596444) ('3075', 0.43357107626700286)
 ('2664', 0.43357107626700286) ('2557', 0.43357107626700286)
 ('2289', 0.43357107626700286) ('1974', 5.4022579149911475e-06)
 ('2971', 4.1467293051285935e-06) ('2534', 4.1467293051285935e-06)
 ('1835', 4.1467293051285935e-06) ('335', 6.969450230556098e-12)]
